In [ ]:
!pip install nibabel

In [ ]:
!pip install matplotlib

In [5]:
import numpy as np
import os
import glob
import nibabel as nib
import sys
import json
import matplotlib.pyplot as plt

In [2]:
import sys
from pathlib import Path
sys.path.insert(0, str(Path(r'C:\Users\ai6\Desktop\git\ANALYSIS-NIFTI-FOR-DEEP-LEARNING\src\preprocessing.py').resolve().parent.parent))
sys.path.insert(0, str(Path(r'C:\Users\ai6\Desktop\git\ANALYSIS-NIFTI-FOR-DEEP-LEARNING\src\helper.py').resolve().parent.parent))

In [9]:
!pip install pandas

In [10]:
from src.helper import *
from src.eda_nifti import *

In [12]:
nii_list = recursive_find_all_files(r'C:\Users\ai6\Desktop\git\ANALYSIS-NIFTI-FOR-DEEP-LEARNING\data\test_data(3person)', 'nii.gz')

In [13]:
nii_list

['C:\\Users\\ai6\\Desktop\\git\\ANALYSIS-NIFTI-FOR-DEEP-LEARNING\\data\\test_data(3person)\\mask\\sub-strokecase0049\\ses-0001\\sub-strokecase0049_ses-0001_msk.nii.gz',
 'C:\\Users\\ai6\\Desktop\\git\\ANALYSIS-NIFTI-FOR-DEEP-LEARNING\\data\\test_data(3person)\\mask\\sub-strokecase0050\\ses-0001\\sub-strokecase0050_ses-0001_msk.nii.gz',
 'C:\\Users\\ai6\\Desktop\\git\\ANALYSIS-NIFTI-FOR-DEEP-LEARNING\\data\\test_data(3person)\\mask\\sub-strokecase0051\\ses-0001\\sub-strokecase0051_ses-0001_msk.nii.gz',
 'C:\\Users\\ai6\\Desktop\\git\\ANALYSIS-NIFTI-FOR-DEEP-LEARNING\\data\\test_data(3person)\\rawdata\\sub-strokecase0049\\ses-0001\\sub-strokecase0049_ses-0001_adc.nii.gz',
 'C:\\Users\\ai6\\Desktop\\git\\ANALYSIS-NIFTI-FOR-DEEP-LEARNING\\data\\test_data(3person)\\rawdata\\sub-strokecase0049\\ses-0001\\sub-strokecase0049_ses-0001_dwi.nii.gz',
 'C:\\Users\\ai6\\Desktop\\git\\ANALYSIS-NIFTI-FOR-DEEP-LEARNING\\data\\test_data(3person)\\rawdata\\sub-strokecase0049\\ses-0001\\sub-strokecase0049

In [40]:
matching_path_dict = dict()

In [37]:
def get_mask_nii_path(nii_list, subject_numb, mask_word):

    mask_nii_path = None
    for nii_path in nii_list:
        if subject_numb in nii_path and mask_word in nii_path:
            mask_nii_path = nii_path

    return mask_nii_path


In [46]:
def get_matching_path_dict(nii_list, maskword): 
    matching_path_dict = dict()
    for nii_path in nii_list:

        if 'msk' not in nii_path:
            raw_nii = nii_path
            
            try:
                subject = nii_path.split('\\')[-1].split('_')[0]
            except:
                subject = nii_path.split('/')[-1].split('_')[0]

            mask_nii_path = get_mask_nii_path(nii_list, subject, maskword)

            matching_path_dict[raw_nii] = mask_nii_path

        else:
            pass
    
    return matching_path_dict

In [48]:
def multiply(x):
    return x*x

def add(x):
    return (x+x)

funcs = [multiply, add]
for i in range(5):
    value = list(map(lambda f: f(i), funcs))

print(value)

[16, 8]


In [44]:
matching_path_dict = get_matching_path_dict(nii_list, 'msk')

In [45]:
matching_path_dict

{'C:\\Users\\ai6\\Desktop\\git\\ANALYSIS-NIFTI-FOR-DEEP-LEARNING\\data\\test_data(3person)\\rawdata\\sub-strokecase0049\\ses-0001\\sub-strokecase0049_ses-0001_adc.nii.gz': 'C:\\Users\\ai6\\Desktop\\git\\ANALYSIS-NIFTI-FOR-DEEP-LEARNING\\data\\test_data(3person)\\mask\\sub-strokecase0051\\ses-0001\\sub-strokecase0051_ses-0001_msk.nii.gz',
 'C:\\Users\\ai6\\Desktop\\git\\ANALYSIS-NIFTI-FOR-DEEP-LEARNING\\data\\test_data(3person)\\rawdata\\sub-strokecase0049\\ses-0001\\sub-strokecase0049_ses-0001_dwi.nii.gz': 'C:\\Users\\ai6\\Desktop\\git\\ANALYSIS-NIFTI-FOR-DEEP-LEARNING\\data\\test_data(3person)\\mask\\sub-strokecase0051\\ses-0001\\sub-strokecase0051_ses-0001_msk.nii.gz',
 'C:\\Users\\ai6\\Desktop\\git\\ANALYSIS-NIFTI-FOR-DEEP-LEARNING\\data\\test_data(3person)\\rawdata\\sub-strokecase0049\\ses-0001\\sub-strokecase0049_ses-0001_flair.nii.gz': 'C:\\Users\\ai6\\Desktop\\git\\ANALYSIS-NIFTI-FOR-DEEP-LEARNING\\data\\test_data(3person)\\mask\\sub-strokecase0051\\ses-0001\\sub-strokecase0051_

In [50]:
!pip install simpleITK

In [65]:
import SimpleITK as sitk

In [66]:
def make_isotropic(image, interpolator = sitk.sitkLinear, spacing = None):
    '''
    Many file formats (e.g. jpg, png,...) expect the pixels to be isotropic, same
    spacing for all axes. Saving non-isotropic data in these formats will result in
    distorted images. This function makes an image isotropic via resampling, if needed.
    Args:
        image (SimpleITK.Image): Input image.
        interpolator: By default the function uses a linear interpolator. For
                      label images one should use the sitkNearestNeighbor interpolator
                      so as not to introduce non-existant labels.
        spacing (float): Desired spacing. If none given then use the smallest spacing from
                         the original image.
    Returns:
        SimpleITK.Image with isotropic spacing which occupies the same region in space as
        the input image.
    '''
    original_spacing = image.GetSpacing()
    # Image is already isotropic, just return a copy.
    if all(spc == original_spacing[0] for spc in original_spacing):
        return sitk.Image(image)
    # Make image isotropic via resampling.
    original_size = image.GetSize()
    if spacing is None:
        spacing = min(original_spacing)
    new_spacing = [spacing]*image.GetDimension()
    new_size = [int(round(osz*ospc/spacing)) for osz, ospc in zip(original_size, original_spacing)]
    return sitk.Resample(image, new_size, sitk.Transform(), interpolator,
                         image.GetOrigin(), new_spacing, image.GetDirection(), 0, # default pixel value
                         image.GetPixelID())

In [67]:
def resample_fixedsize_fixedspacing (nifti_file_path, save_folder_path, interpolation, new_size, new_spacing):
    '''
    #img_sitk : sitk image (3D)
    #interppolation: sitk.sitkLinear
    #new_size : 64
    #new_spacing: [1,1,1]
    
    (example)
    new_img = resample_fixedsize_fixedspacing(nii_list[0], './result', sitk.sitkLinear, 64, [1,1,1])
    '''


    img_sitk = sitk.ReadImage(nifti_file_path)
    # img_array = sitk.GetArrayFromImage(img_sitk)
    try:
        file_name = nifti_file_path.split('\\')[-1].rstrip('.nii.gz')
    except:
        file_name = nifti_file_path.split('/')[-1].rstrip('.nii.gz')

    dimension = img_sitk.GetDimension()
    reference_size = [new_size,new_size,new_size]
    reference_physical_size = np.zeros(img_sitk.GetDimension())
    reference_physical_size[:] = [(sz-1)*spc if sz*spc>mx  else mx for sz,spc,mx in zip(img_sitk.GetSize(), img_sitk.GetSpacing(), reference_physical_size)]
    reference_direction = img_sitk.GetDirection()
    reference_origin = img_sitk.GetOrigin()
    
    reference_image = sitk.Image(reference_size, img_sitk.GetPixelIDValue())
    reference_image.SetOrigin(reference_origin)
    reference_image.SetSpacing(new_spacing)
    reference_image.SetDirection(reference_direction)

    reference_center = np.array(reference_image.TransformContinuousIndexToPhysicalPoint(np.array(reference_image.GetSize())/2.0))
    
    transform = sitk.AffineTransform(dimension)
    transform.SetMatrix(img_sitk.GetDirection())
    transform.SetTranslation(np.array(img_sitk.GetOrigin()) - reference_origin)
  
    centering_transform = sitk.TranslationTransform(dimension)
    img_center = np.array(img_sitk.TransformContinuousIndexToPhysicalPoint(np.array(img_sitk.GetSize())/2.0))
    centering_transform.SetOffset(np.array(transform.GetInverse().TransformPoint(img_center) - reference_center))
    centered_transform = sitk.CompositeTransform([centering_transform,transform])
    new_img = sitk.Resample(img_sitk, reference_image, centered_transform, interpolation, 0.0)
    
    os.makedirs(save_folder_path, exist_ok=True)
    sitk.WriteImage(new_img, f'{save_folder_path}/{file_name}_resampled.nii.gz')

    return new_img

In [68]:
nii_list[0]

'C:\\Users\\ai6\\Desktop\\git\\ANALYSIS-NIFTI-FOR-DEEP-LEARNING\\data\\test_data(3person)\\mask\\sub-strokecase0049\\ses-0001\\sub-strokecase0049_ses-0001_msk.nii.gz'

In [69]:
new_img = resample_fixedsize_fixedspacing(nii_list[0], './result', sitk.sitkLinear, 64, [1,1,1])

In [70]:
############# 모든 테스트 파일을 대상으로 resampling 수행
###  src.resampling의 resample_fixedsize_fixedspacing 사용

In [72]:
[resample_fixedsize_fixedspacing(nii_file, './result', sitk.sitkLinear, 128, [1,1,1]) for nii_file in nii_list]

[<SimpleITK.SimpleITK.Image; proxy of <Swig Object of type 'std::vector< itk::simple::Image >::value_type *' at 0x000001B2318AF660> >,
 <SimpleITK.SimpleITK.Image; proxy of <Swig Object of type 'std::vector< itk::simple::Image >::value_type *' at 0x000001B2318AF870> >,
 <SimpleITK.SimpleITK.Image; proxy of <Swig Object of type 'std::vector< itk::simple::Image >::value_type *' at 0x000001B2318AFBA0> >,
 <SimpleITK.SimpleITK.Image; proxy of <Swig Object of type 'std::vector< itk::simple::Image >::value_type *' at 0x000001B2318AFC30> >,
 <SimpleITK.SimpleITK.Image; proxy of <Swig Object of type 'std::vector< itk::simple::Image >::value_type *' at 0x000001B2318AF840> >,
 <SimpleITK.SimpleITK.Image; proxy of <Swig Object of type 'std::vector< itk::simple::Image >::value_type *' at 0x000001B2318AFCC0> >,
 <SimpleITK.SimpleITK.Image; proxy of <Swig Object of type 'std::vector< itk::simple::Image >::value_type *' at 0x000001B2318AF330> >,
 <SimpleITK.SimpleITK.Image; proxy of <Swig Object of t